<a href="https://colab.research.google.com/github/ankushkhakale/-Sign-Language-Recognition-with-PyTorch-CNN/blob/main/ASL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# --- Data Setup (to resolve FileNotFoundError) ---
import os
import zipfile
import shutil

# 1. Install Kaggle library
!pip install kaggle --quiet

# 2. Set up Kaggle API (User needs to upload kaggle.json)
# Important: Please upload your kaggle.json file to the Colab environment.
# Go to Files (left sidebar) -> .kaggle folder (create if not exists: !mkdir -p ~/.kaggle)
# Then upload kaggle.json there.
# Ensure correct permissions: !chmod 600 ~/.kaggle/kaggle.json

# If you have already uploaded kaggle.json and set permissions, you can comment out the above instructions.

# Define dataset path and name
KAGGLE_DATASET_NAME = "grassknoted/asl-alphabet" # Example ASL Alphabet dataset on Kaggle
DOWNLOAD_PATH = "./data_download" # Temporary download path
TARGET_ROOT = "./data/asl_alphabet" # Final structure root for ImageFolder

# Create necessary directories
os.makedirs(DOWNLOAD_PATH, exist_ok=True)
os.makedirs(TARGET_ROOT, exist_ok=True) # Ensure data/asl_alphabet exists

# 3. Download the dataset from Kaggle
print(f"Downloading {KAGGLE_DATASET_NAME} dataset...")
# Using a try-except block in case kaggle.json is not set up correctly
try:
    !kaggle datasets download {KAGGLE_DATASET_NAME} -p {DOWNLOAD_PATH}
    print("Download complete.")
except Exception as e:
    print(f"Error downloading dataset: {e}")
    print("Please ensure your Kaggle API key (kaggle.json) is properly configured in ~/.kaggle.")
    # Raise the error to stop execution if download fails
    raise FileNotFoundError("Kaggle dataset download failed. Check API key and internet connection.")

# 4. Unzip the dataset
# Kaggle typically downloads a single zip for datasets. Assume 'asl-alphabet.zip'
zip_file_path = os.path.join(DOWNLOAD_PATH, 'asl-alphabet.zip')

# Check for alternative zip file names if 'asl-alphabet.zip' isn't found
if not os.path.exists(zip_file_path):
    zip_files = [f for f in os.listdir(DOWNLOAD_PATH) if f.endswith('.zip')]
    if zip_files:
        zip_file_path = os.path.join(DOWNLOAD_PATH, zip_files[0])
    else:
        raise FileNotFoundError(f"No zip file found in {DOWNLOAD_PATH} after download.")

print(f"Unzipping {zip_file_path}...")
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(DOWNLOAD_PATH) # Extract all into the download path
print("Unzipping complete.")

# 5. Restructure the dataset to match torchvision.datasets.ImageFolder expectation
# The unzipped structure might be: ./data_download/asl_alphabet_train/A/...
# We need: ./data/asl_alphabet/train/A/...

source_train_dir = os.path.join(DOWNLOAD_PATH, 'asl_alphabet_train')
target_train_dir = os.path.join(TARGET_ROOT, 'train')

source_test_dir = os.path.join(DOWNLOAD_PATH, 'asl_alphabet_test')
target_test_dir = os.path.join(TARGET_ROOT, 'test')

# Move 'asl_alphabet_train' to 'data/asl_alphabet/train'
if os.path.exists(source_train_dir):
    print(f"Moving '{source_train_dir}' to '{target_train_dir}'...")
    shutil.move(source_train_dir, target_train_dir)
else:
    print(f"Warning: '{source_train_dir}' not found. Check unzipped directory structure.")

# Move 'asl_alphabet_test' to 'data/asl_alphabet/test'
if os.path.exists(source_test_dir):
    print(f"Moving '{source_test_dir}' to '{target_test_dir}'...")
    shutil.move(source_test_dir, target_test_dir)
else:
    print(f"Warning: '{source_test_dir}' not found. Check unzipped directory structure.")

# Clean up temporary download folder (optional)
# shutil.rmtree(DOWNLOAD_PATH)
print("Dataset preparation complete.")
# --- End Data Setup ---

import torch
import torch.nn as nn
import torchvision
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

# Transformations
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.Grayscale(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Load ASL Alphabet Dataset (e.g., ASL Alphabet dataset from Kaggle)
# Directory structure: data/asl_alphabet/A/, B/, ..., Z/
train_dataset = datasets.ImageFolder(root='./data/asl_alphabet/train', transform=transform)
test_dataset = datasets.ImageFolder(root='./data/asl_alphabet/test', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64)

# CNN Model
class SignLanguageCNN(nn.Module):
    def __init__(self, num_classes=26):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(64 * 16 * 16, 256),
            nn.ReLU(),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        return self.net(x)

model = SignLanguageCNN(num_classes=26)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(5):
    for images, labels in train_loader:
        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

# Evaluation
correct, total = 0, 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        preds = torch.argmax(outputs, dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

print(f"Test Accuracy: {100 * correct / total:.2f}%")

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 10, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/kaggle/cli.py", line 68, in main
    out = args.func(**command_args)
          ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/kaggle/api/kaggle_api_extended.py", line 1741, in dataset_download_cli
    with self.build_kaggle_client() as kaggle:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/kaggle/api/kaggle_api_extended.py", line 688, in build_kaggle_client
    username=self.config_values['username'],
             ~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^
KeyError: 'username'
Download complete.


FileNotFoundError: No zip file found in ./data_download after download.